TODO:

* [ ] create GD/SGD
* [ ] Score w/ loss

In [1]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import numpy as np

In [2]:
from tune import Damper
from pprint import pprint
import pandas as pd

In [3]:
from distributed import Client, LocalCluster
# client = await Client("localhost:6786", asynchronous=True)
# cluster = await LocalCluster(n_workers=3, threads_per_worker=2, asynchronous=True)
# client = await Client(cluster)
# client = await Client("localhost:8786", asynchronous=True)
client = await Client(asynchronous=True)
# client = Client()
client

Client Scheduler: tcp://127.0.0.1:53516 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [4]:
await client.upload_file("train.py")
await client.upload_file("tune.py")

In [5]:
def test():
    from tune import Damper
    d = Damper()
    return True

f = client.submit(test)
res = await client.gather(f)
res

True

In [6]:
X, y = fetch_covtype(return_X_y=True, shuffle=False, random_state=0)

In [7]:
X.shape

(581012, 54)

In [8]:
kernel = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_poly = kernel.fit_transform(X)

In [9]:
X_poly.shape

(581012, 1485)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_poly, y, random_state=42, train_size=200_000
)

In [11]:
X_train.nbytes / 1024**3

2.212822437286377

In [12]:
X_train.shape

(200000, 1485)

In [13]:
def _get_cnts_cols(X):
    cols = range(X.shape[1])
    uniqs = [np.unique(X[:, c]) for c in cols]
    cnts = [c for c, _u in zip(cols, uniqs) if len(_u) > 2]
    discrete_cols = [c for c, _u in zip(cols, uniqs) if len(_u) == 2]
    return cnts, discrete_cols

def normalize(X, scale, cnts, discrete):
    Y = scale.transform(X[:, cnts])
    Y2 = X[:, discrete].astype(bool).astype(int)  # one element is 30 (not 0/1)
    Z = np.hstack((Y2, Y))
    return Z

cnts, discrete = _get_cnts_cols(X_train)
scale = StandardScaler().fit(X_train[:, cnts])

print(X_train[:, cnts + discrete].shape)
X_train = normalize(X_train, scale, cnts, discrete)
X_test = normalize(X_test, scale, cnts, discrete)
print(X_train.shape)

uniqs = np.unique(X_train[:, :len(discrete)])
assert len(uniqs) == 2 and 0 <= uniqs.min() <= uniqs.max() <= 1

(200000, 613)
(200000, 613)


In [14]:
uniqs

array([0., 1.])

In [15]:
X_train.shape

(200000, 613)

In [16]:
X_test.shape

(381012, 613)

In [17]:
from scipy.stats import loguniform, uniform
from copy import copy

batch_pow = np.arange(5, 9 + 1).astype(int)
static_batch = (2 ** batch_pow).astype(int).tolist()

ibs = (2 ** batch_pow).astype(int).tolist()
mbs = (2 ** (batch_pow + 1)).astype(int).tolist()

lrs = loguniform(0.5e-4, 1e-1)
weight_decays = loguniform(1e-8, 1e-2)
momentums = uniform(0, 1)

In [32]:
base = {
    "max_batch_size": static_batch,  # 5 choices
    "weight_decay": weight_decays,  # cnts
    "momentum": momentums,  # cnts
    "lr": [0.9e-3],
    "scoring": ["loss"],
}

dwell = [1, 2, 5, 10, 20, 50, 100, 200]
padadamp_params = {
    **copy(base),
    "batch_growth_rate": loguniform(1e-3, 1e-1),
    "dwell": dwell,
    "initial_batch_size": ibs,
    "max_batch_size": mbs,
}

hsgd_params = {
    **copy(base),
    "initial_batch_size": [256],
    "max_batch_size": [2048],
    "lr": [0.9e-3],
    "batch_growth_rate": loguniform(1e-3, 1e-1),
}

In [33]:
gd_params = {
    **copy(base),
    "max_batch_size": [len(X_train)],
}

In [34]:
gd_params

{'max_batch_size': [200000],
 'weight_decay': <scipy.stats._distn_infrastructure.rv_frozen at 0x129f319d0>,
 'momentum': <scipy.stats._distn_infrastructure.rv_frozen at 0x11b46b1c0>,
 'lr': [0.0009],
 'scoring': ['loss']}

In [35]:
import numpy as np
n_examples = 75 * len(X_train)
n_params = 25

max_iter = n_params
chunk_size = np.round(n_examples / n_params) + 0
chunk_size / len(X_train)

3.0

In [48]:
import numpy as np
n_examples = 200 * len(X_train)
n_params = 200

max_iter = n_params
chunk_size = np.round(n_examples / n_params) + 0
chunk_size = min(len(X_train), chunk_size)
chunk_size / len(X_train)

1.0

In [49]:
# import dask.array as da
# max_iter = 27
# chunk_size = 10_000

# n = 3000
# X_train = np.random.uniform(size=(n, 54))
# y_train = np.random.choice(8, size=n)

In [50]:
import dask.array as da
X_train2 = da.from_array(X_train, chunks=(chunk_size, -1))
y_train2 = da.from_array(y_train, chunks=chunk_size)

X_train2.chunks

((200000,), (613,))

In [40]:
X_train2 = X_train2.persist()
y_train2 = y_train2.persist()

In [41]:
del X_test
del y_test

NameError: name 'X_test' is not defined

In [43]:
from tune import PadaDamp, Damper, GD, HSGD

seed = 33

padadamp = PadaDamp(seed=seed)
hsgd = HSGD(seed=seed, name="hsgd")
gd = GD(seed=seed, name="gd")
sgd = Damper(seed=seed, name="sgd")
asgd = Damper(seed=seed, name="asgd", opt="asgd")

In [44]:
from sklearn.base import clone
p2 = clone(padadamp)
p2.initialize()
n_params = [v.nelement() for v in p2.model.module_.parameters()]
sum(n_params)

4912

In [45]:
from dask_ml.model_selection import HyperbandSearchCV

kwargs = dict(
    max_iter=max_iter, verbose=True, random_state=6, patience=2, tol=None,
)

models = {
#     "pada": (padadamp, padadamp_params),
#     "gd": (gd, gd_params),
#     "asgd": (sgd, base),
    "hsgd": (hsgd, hsgd_params),
}

searches = {
    name: HyperbandSearchCV(model, params, prefix=f"-{name}", **kwargs)
    for name, (model, params) in models.items()
}

In [46]:
# _search = searches["pada"]
_search = searches["hsgd"]
{k: v for k, v in _search.metadata.items() if not isinstance(v, list)}

{'partial_fit_calls': 173, 'n_models': 17}

In [47]:
X_train2.shape

(200000, 613)

In [33]:
# jobs = {name: search.fit(X_train2, y_train2) for name, search in searches.items()}
import asyncio
jobs = [
    search.fit(X_train2, y_train2) if name != "gd" else search.fit(X_train, y_train)
    for name, search in searches.items()
]

In [30]:
running_searches = asyncio.gather(*jobs)

[CV-geo, bracket=5] creating 243 models
[CV-geo, bracket=4] creating 98 models
[CV-geo, bracket=3] creating 41 models
[CV-geo, bracket=2] creating 18 models
[CV-geo, bracket=1] creating 9 models
[CV-geo, bracket=0] creating 6 models
[CV-pada, bracket=5] creating 243 models
[CV-pada, bracket=4] creating 98 models
[CV-pada, bracket=3] creating 41 models
[CV-pada, bracket=2] creating 18 models
[CV-pada, bracket=1] creating 9 models
[CV-pada, bracket=0] creating 6 models
[CV-gd, bracket=5] creating 243 models
[CV-gd, bracket=4] creating 98 models
[CV-gd, bracket=3] creating 41 models
[CV-gd, bracket=2] creating 18 models
[CV-gd, bracket=1] creating 9 models
[CV-gd, bracket=0] creating 6 models
[CV-sgd, bracket=5] creating 243 models
[CV-sgd, bracket=4] creating 98 models
[CV-sgd, bracket=3] creating 41 models
[CV-sgd, bracket=2] creating 18 models
[CV-sgd, bracket=1] creating 9 models
[CV-sgd, bracket=0] creating 6 models
[CV-adagrad, bracket=5] creating 243 models
[CV-adagrad, bracket=4] 

In [ ]:
running_searches

In [27]:
finished_searches = running_searches.result()

In [28]:
finished_searches

[HyperbandSearchCV(estimator=GeoDamp(seed=33), max_iter=300,
                   parameters={'dampingdelay': [1, 2, 5, 10, 20, 50, 100],
                               'dampingfactor': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1155e2640>,
                               'initial_batch_size': [32, 64, 128, 256, 512],
                               'lr': <scipy.stats._distn_infrastructure.rv_frozen object at 0x115734400>,
                               'max_batch_size': [64, 128, 256, 512, 1024],
                               'momentum': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1157347c0>,
                               'scoring': ['loss'],
                               'weight_decay': <scipy.stats._distn_infrastructure.rv_frozen object at 0x115734460>},
                   patience=True, prefix='-geo', random_state=6, verbose=True),
 HyperbandSearchCV(estimator=PadaDamp(seed=33), max_iter=300,
                   parameters={'batch_growth_rate': <scipy.stats

In [29]:
for finished_search in finished_searches:
    name = "data/" + finished_search.prefix[1:]
    pd.DataFrame(finished_search.cv_results_).to_csv(f"{name}-cv-results.csv", index=False)
    pd.DataFrame(finished_search.history_).to_csv(f"{name}-hist.csv", index=False)
    pd.DataFrame(finished_search.best_estimator_.history_).to_csv(f"{name}-best-est-hist.csv", index=False)
    with open(f"{name}-best-params.json", "w") as f:
        json.dump(finished_search.best_params_, f)
